In [ ]:
# variables for the ETL run

# Set to true when trying to refresh the coverage curves on the Database
#  it is only needed once per quarter.
upload_to_drive_csv = True 

# Set it to true to store the results of the coverage fitting process in the target
# gsheet
save_to_gsheets_and_csv = True


# Reload data from database
# if set to true the etl will get new data from snowflake.
# if false, the notebook will read a local csv file
load_db_data = True


In [ ]:
import subprocess
import sys
import os as os

reqs = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'])
installed_packages = [r.decode().split('==')[0] for r in reqs.split()]

# only run if packages are not installed
if 'pygsheets' not in installed_packages:
    !{sys.executable} -m pip install  pygsheets
    !{sys.executable} -m pip install "pyarrow<5.1.0,>=5.0.0;"
    !{sys.executable} -m pip install  pydrive

import pygsheets
import configparser
import sys
import snowflake.connector
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np                  
import matplotlib.pyplot as plt     
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
import seaborn as sns
from math import floor 
from scipy.optimize import curve_fit

In [ ]:
# set base permission files and snowflake engine

# link to snowflake library
from gitlabdata.orchestration_utils import (
    data_science_engine_factory,
    query_dataframe,
)

engine = data_science_engine_factory()


#target gsheet for uploads and downloads, just allowing one to keep it more secure
# x-ray source target id
x_ray_spreadsheet_id = '1Vwu8euxRgIF3QYWK8hAbp4Vy21AlFfpDwI4MaEEiIWk'

# the following gsheets are used to push the data into the snowflake
curves_sheetload_spreadsheet_id = '1dLevdYA8QMjpIV9irNGD8KTfKIgc3xJdrRDA2vH7M50'

#key to be able to work with bigquery / gsheets
service_file_path = '/Users/nfiguera/repos/sales-strategy-and-analytics-business-intelligence/202202_fy22_channel_value_analysis/nfiguera-c3fe9e64-a6543dd51e79.json'


os.getcwd()

# NF: Just to deal with my working directory changing
# Get the current working directory
cwd = os.getcwd()
print(cwd)
os.chdir("/Users/nfiguera/repos/sales-strategy-and-analytics-business-intelligence/202202_targets_&_coverage_curves")

# Model Description:

## Goal

Retrieve the quarterly coverage raw data & calculate fitted coverage curves for each level of aggregation
 
## Steps

- Retrieve data metrics per day models
- Calculate the quarters needed for each metric
- Integrate the different cuts into a single tall table
- Pre-aggregate the data by business cut & calculate coverage
- Using pre-aggregated data, fit a curve for each pre-aggregated model
- Push changes to gSheet

## Challenges with automation

- How can we authenticate on gSheets without using my own auth json. (Maybe a service account within the library)?
- How can we push data into Snowflake, ideally the workspace_sales directly from the jupyter workbook?


## Update to FY / 4Q historical curves

List of steps:
- Create metric per day for FY
- How to create a metric day perspective for 4Q cuts?
  - Starting quarter and end quarter
  - Total booked on that time period
  - Day 365 is the last day of end quarter
 


import pydrive

from pydrive.drive import GoogleDrive

gauth = pydrive.auth.GoogleAuth(service_file_path).LoadClientConfigFile(service_file_path)
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':'https://drive.google.com/file/d/1RIuSxxpd3Q_2bfpxPng5fu3Oaa3wqC0j'}) # replace the id with id of the file you want to access
downloaded.GetContentFile('file.csv')  
GoogleAuth(service_file_path)
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':'https://drive.google.com/file/d/1RIuSxxpd3Q_2bfpxPng5fu3Oaa3wqC0j'}) # replace the id with id of the file you want to access
downloaded.GetContentFile('file.csv')  

# Helper Functions

In [ ]:
#######
## Create a fitter curve of the last 4 quarters for comparison
#######
def objective(x, a, b, c, d, e):
    return a * x + b * x**2 + c * x**3 + d * x**4 + e


def write_to_gsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1

def read_from_gsheet(service_file_path, spreadsheet_id, sheet_name):
    """
    this function takes a sheet_name from a spreadsheet_id and returns a data frame 
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
   
    wks_read = sh.worksheet_by_title(sheet_name)
    read = wks_read.get_as_df()
    
    return read


def run_query_in_snowflake(conn, sql):
    cur = conn.cursor()
    cur.execute(sql)
    df = cur.fetch_pandas_all()
    return df

def executeScriptFromFile(filename, engine):
    # Open and read the file as a single buffer
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()
    
    print(filename)
    print(len(sqlFile))
    
    results = -1
    
    try:
        results = query_dataframe(engine,sqlFile)
    except:
        print("Command did not run")
    
    return results

###########################################################

def days_between(d1, d2):
    #d1 = datetime.strptime(d1, "%Y-%m-%d")
    #d2 = datetime.strptime(d2, "%Y-%m-%d")
    return (d2 - d1).days


def calculate_quarters_after_creation(x):
        
    age = 0
    
    if (x['IS_OPEN'] == 1):
        age = days_between(x['CREATED_FISCAL_QUARTER_DATE'], x['SNAPSHOT_FISCAL_QUARTER_DATE'])
    elif (x['IS_OPEN']== 0 and x['SNAPSHOT_DATE'] <= x['CLOSE_DATE']):
        age = days_between(x['CREATED_FISCAL_QUARTER_DATE'], x['SNAPSHOT_FISCAL_QUARTER_DATE'])
    else:
        age = days_between(x['CREATED_FISCAL_QUARTER_DATE'], x['CLOSE_FISCAL_QUARTER_DATE'])
    
    quarter_delta = floor(age/90)
    
    return quarter_delta

def calculate_channel_track (x):
    
    channel_track = 'Direct'
    
    if (x['deal_path'] == 'Direct'):
        channel_track = 'Direct'
    elif (x['deal_path'] == 'Web Direct'):
        channel_track = 'Web Direct'
    elif (x['deal_path'] == 'Channel'
        and x['sales_qualified_source'] != 'Channel Generated'): 
        channel_track = 'Partner Co-Sell'
    elif (x['deal_path'] == 'Channel'):
        channel_track = 'Partner Sourced'
    
    return channel_track

In [ ]:
# calculates pending coverage using a minimum of 50k pending
def calculate_to_pending_coverage (x, metric, qtd, actual, minimum_delta=5000):
    
    result = None
    
    actual = float(x[actual])
    qtd = float(x[qtd])
    metric = float(x[metric])
    
    if (actual - qtd) > minimum_delta:
        result = metric / (actual - qtd)
        result = min(result,6) # limiting the maximum amount of coverage to account for noise in the models
    
    return result

# calculates pending coverage using a minimum of 50k pending
def calculate_bookings_linearity (x, qtd_bookings, actual_booked):
    
    result = None
    
    actual = float(x[actual_booked])
    qtd = float(x[qtd_bookings])
       
    if actual > 0:
        result = qtd / actual 
        
    return result 


# fits a curve to the subset data using the defined objective function
def fit_curve_to_agg (data_agg, x_label, y_label):
    
    # fit a curve
    x, y = data_agg[x_label], data_agg[y_label]
    # curve fit
    popt, _ = curve_fit(objective, x, y, method='dogbox')

    x_line = np.arange(min(x), max(x), 1)
    # calculate the output for the range
    # summarize the parameter values
    a, b, c , d, e = popt
    y_line = objective(x_line, a, b, c, d, e)
  
    curve_result = pd.DataFrame({x_label:x_line,y_label:y_line})
    return curve_result

## Data Collection

In [ ]:

### considered keys
agg_key_list = ['key_segment','key_overall','sales_team_rd_asm_level',
       'key_sqs', 'key_ot','key_segment_sqs', 'key_segment_ot', 'key_segment_geo', 'key_segment_geo_sqs',
       'key_segment_geo_ot', 'key_segment_geo_region',
       'key_segment_geo_region_sqs', 'key_segment_geo_region_ot',
       'key_segment_geo_region_area', 'key_segment_geo_region_area_sqs','key_ent_comm',
       'key_segment_geo_region_area_ot','report_user_segment_geo_region_area'] #

In [ ]:
# Collect quarterly data
sql_file = 'qtr_metrics_by_day'

#load_db_data = False

if load_db_data: 
    print ("Executing {} Snowflake SQL".format(sql_file))
    qtr_metrics_by_day = executeScriptFromFile(sql_file+'.sql', engine)
    qtr_metrics_by_day.to_csv(sql_file+'.csv',index=False)
else:
    print ("Reading Local CSV {}.csv".format(sql_file))
    qtr_metrics_by_day = pd.read_csv(sql_file+'.csv')

In [ ]:
# extracted quarters
qtr_metrics_by_day.close_fiscal_quarter_name.sort_values().unique()

# Data Transformation

We collect the data and identify the quarters that we need for each metric.

The way the model is construct, it will have for any given quarter values for open pipeline closing in the same quarter and in future quarters.

To calculate the Current Quarter + 1 & + 2 metrics, we use the specific future field and filter the close quarter to the right point in time


In [ ]:
# transform potential strings to date (in case of data read from CSV)
qtr_metrics_by_day['close_fiscal_quarter_date'] = pd.to_datetime(qtr_metrics_by_day['close_fiscal_quarter_date']).dt.date
qtr_metrics_by_day['rq_plus_1_close_fiscal_quarter_date'] = pd.to_datetime(qtr_metrics_by_day['rq_plus_1_close_fiscal_quarter_date']).dt.date
qtr_metrics_by_day['rq_plus_2_close_fiscal_quarter_date'] = pd.to_datetime(qtr_metrics_by_day['rq_plus_2_close_fiscal_quarter_date']).dt.date

#################
##### Data Injections

# create an artificial global key to calculate a global curve for the whole company
qtr_metrics_by_day['key_overall'] = 'global'

# adjust the sales_team legacy key to be sure that it won't generate duplicate with the FY23 key logic
qtr_metrics_by_day['sales_team_rd_asm_level'] = 'st_rd_' + qtr_metrics_by_day['sales_team_rd_asm_level']

# add a commercial and enterprise consolidated key
#add key enterprise commercial for x-ray reporting
qtr_metrics_by_day['key_ent_comm'] = 'other'
qtr_metrics_by_day.loc[qtr_metrics_by_day['key_segment']=='large','key_ent_comm'] = 'enterprise'
qtr_metrics_by_day.loc[qtr_metrics_by_day['key_segment']=='pubsec','key_ent_comm'] = 'enterprise'
qtr_metrics_by_day.loc[qtr_metrics_by_day['key_segment']=='mid-market','key_ent_comm'] = 'commercial'
qtr_metrics_by_day.loc[qtr_metrics_by_day['key_segment']=='smb','key_ent_comm'] = 'commercial'

#################

# identify the quarters that will be considered when creating the curves
# for current quarter last 4 quarters
index_cond = (qtr_metrics_by_day['close_fiscal_quarter_date'] <= date.today() + relativedelta(months=-3)) & (qtr_metrics_by_day['close_fiscal_quarter_date'] >= date.today() + relativedelta(months=-15))
cq_considered_quarters = qtr_metrics_by_day[index_cond].close_fiscal_quarter_name.sort_values().unique()

# for current quarter + 1, between 5 and 1 quarter ago (as we need the total won amount of the quarter to calculate coverage)
index_cond = (qtr_metrics_by_day['close_fiscal_quarter_date'] <= date.today() + relativedelta(months=-6)) & (qtr_metrics_by_day['close_fiscal_quarter_date'] >= date.today() + relativedelta(months=-18))
cq_1plus_considered_quarters = qtr_metrics_by_day[index_cond].close_fiscal_quarter_name.sort_values().unique()

# for current quarter + 2, between 6 and 2 quarter ago (as we need the total won amount of the quarter to calculate coverage)
index_cond = (qtr_metrics_by_day['close_fiscal_quarter_date'] <= date.today() + relativedelta(months=-9)) & (qtr_metrics_by_day['close_fiscal_quarter_date'] >= date.today() + relativedelta(months=-21))
cq_2plus_considered_quarters = qtr_metrics_by_day[index_cond].close_fiscal_quarter_name.sort_values().unique()

print(cq_considered_quarters)
print(cq_1plus_considered_quarters)
print(cq_2plus_considered_quarters)

In [ ]:
# separate considered data points per metric type
cq_metrics_data = qtr_metrics_by_day[qtr_metrics_by_day['close_fiscal_quarter_name'].isin(cq_considered_quarters)].copy()
cq_plus1_metrics_data = qtr_metrics_by_day[qtr_metrics_by_day['close_fiscal_quarter_name'].isin(cq_1plus_considered_quarters)].copy()
cq_plus2_metrics_data = qtr_metrics_by_day[qtr_metrics_by_day['close_fiscal_quarter_name'].isin(cq_2plus_considered_quarters)].copy()



## From a wide table to a tall table

After selecting and appending the relevant datasets into a single data with the same column name structure we unpivot the table into a tall one with a line per metric per day per fiscal quarter.

In [ ]:
# There are three fields that are critical
# - Open Pipe Net ARR (Stage 1, 3, 4) (These can be metrics in a tall table)
# - Booked Amount (This could be a column)
# - Total Booked Amount (This can also be a column)


### current quarter
variables = ['close_day_of_fiscal_quarter_normalised','close_fiscal_quarter_name','total_booked_net_arr','booked_net_arr'] + agg_key_list
considered_metrics = ['open_1plus_net_arr','open_3plus_net_arr','open_4plus_net_arr'] 
cq_melt = cq_metrics_data.melt(id_vars = variables, value_vars = considered_metrics)

### current quarter + 1 
variables = ['close_day_of_fiscal_quarter_normalised','close_fiscal_quarter_name','rq_plus_1_total_booked_net_arr','booked_net_arr'] + agg_key_list 
considered_metrics = ['rq_plus_1_open_1plus_net_arr', 'rq_plus_1_open_3plus_net_arr',
       'rq_plus_1_open_4plus_net_arr']
cq_plus1_melt = cq_plus1_metrics_data.melt(id_vars = variables, value_vars = considered_metrics)
# rename the total booked field to keep it consistent
cq_plus1_melt = cq_plus1_melt.rename({'rq_plus_1_total_booked_net_arr':'total_booked_net_arr'}, axis=1)
cq_plus1_melt['booked_net_arr'] = 0

### current quarter + 2
variables = ['close_day_of_fiscal_quarter_normalised','close_fiscal_quarter_name','rq_plus_2_total_booked_net_arr','booked_net_arr'] + agg_key_list
considered_metrics = ['rq_plus_2_open_1plus_net_arr', 'rq_plus_2_open_3plus_net_arr',
       'rq_plus_2_open_4plus_net_arr']
cq_plus2_melt = cq_plus2_metrics_data.melt(id_vars = variables, value_vars = considered_metrics)
# rename the total booked field to keep it consistent
cq_plus2_melt = cq_plus2_melt.rename({'rq_plus_2_total_booked_net_arr':'total_booked_net_arr'}, axis=1)
cq_plus2_melt['booked_net_arr'] = 0

### consolidated dataset
combined_df = pd.concat([cq_melt, cq_plus1_melt,cq_plus2_melt], ignore_index=True)
combined_df = combined_df.rename({'variable':'metric_name'}, axis=1)
combined_df = combined_df.rename({'value':'metric_value'}, axis=1)

##########################################################
# to be able to calculate coverage, we need to drop:
# - Lines where total booked net arr = 0
# - Lines where value is NaN or 0
# - Lines where booked_net_arr is NaN are to be set to 0

###
# - Lines where total booked net arr = 0
# - Lines where value is NaN or 0
combined_df.dropna(subset=['total_booked_net_arr', 'metric_value'], inplace=True)

# - Lines where booked_net_arr is NaN are to be set to 0
# combined_df = combined_df.drop(combined_df[combined_df['total_booked_net_arr']==0].index)
len(combined_df)

# test result
print(len(combined_df), ' # of rows - ', combined_df.columns)

## Calculation of coverages per Business cut

For each business cut we need to do an aggregation of the relevant fields and from there run the coverage calculation

In [ ]:
# aggregation columns
agg_columns = ['metric_name','close_day_of_fiscal_quarter_normalised', 'close_fiscal_quarter_name']
# fields to be summarized
agg_fields = ['metric_value','total_booked_net_arr', 'booked_net_arr']

# initialize to none, add. intermediate dataframes
# the goal is to calcualte the coverage for each aggregation key. First the data needs to be grouped
# at that level and the relevant fields summed up. Then the coverage calculation function is used
combined_agg_results = None
for agg_key in agg_key_list:
    
    # the key for aggregation needs to be complimented by the day key and the quarter key
    total_agg_keys = [agg_key] + agg_columns

    #calculate the aggregation per key
    temp = combined_df.groupby(total_agg_keys,dropna=False)[agg_fields].agg('sum').reset_index().copy()
        
    # calculate coverages per metric, all are calculated using the "Pending" approach
    # this needs to be done for every iteration 
    temp['metric_coverage'] = temp.apply(calculate_to_pending_coverage, axis=1,metric='metric_value',qtd='booked_net_arr',actual='total_booked_net_arr')

    # clean up / add information regarding the agg key
    temp = temp.rename({agg_key:'agg_key_value'},axis=1)
    temp['agg_key_name'] = agg_key

    #consolidate results
    if combined_agg_results is None:
        combined_agg_results = temp.copy()
    else:
        combined_agg_results = combined_agg_results.append(temp) 
        

# Calculation of Curve Fit

The previous step created a combined result data where we have a pre-aggregated and precalculated coverage for each value of the distinct aggregations keys.

To use this dataset we must filter by the value of the key we want to use e.g. Large and plot the metric_coverage field. 

The dataset has 4 data points per metric, as we consider 4 past quarters.

In [ ]:
### Curve Fitting per Cut
# for this to run I need to filter a) they aggregation level, b) the specific value at that aggregation level.
# the result will be a table like key_agg, day_metric, 

#initialization
combined_fitted_results = None

#iterate through all the aggregated keys
for agg_key_name in combined_agg_results.agg_key_name.unique():

    #filter temporary data
    temp_key_data = combined_agg_results[combined_agg_results['agg_key_name']==agg_key_name].copy()
    
    #iterate through the unique values of the specific aggregation
    for agg_key_value in temp_key_data.agg_key_value.unique():
        
        # filter specific value per agg temporary data
        temp_key_value_data = temp_key_data[temp_key_data['agg_key_value']==agg_key_value].copy()
                
        # set up the range of the quarter curves we want to report on
        metrics_temp = pd.DataFrame({"close_day_of_fiscal_quarter_normalised":range(0,91)})

        for metric_name in temp_key_value_data.metric_name.unique():
            
            # drop nas
            temp_metric_data = temp_key_value_data[temp_key_value_data['metric_name']==metric_name].copy()
            temp_metric_data.dropna(subset=['metric_coverage'], inplace=True)
            temp_cuve = None
            # avoid fitting curves for cuts that do not have enough data
            if len(temp_metric_data) >= 90:     
                temp_curve = fit_curve_to_agg (temp_metric_data,'close_day_of_fiscal_quarter_normalised','metric_coverage') 
                temp_curve.rename({'metric_coverage':metric_name+'_coverage'},inplace=1,axis=1)   
                metrics_temp =  metrics_temp.merge(temp_curve, how='left', on='close_day_of_fiscal_quarter_normalised')                 
            
        # add the metric detail to the dataset
        metrics_temp['agg_key_name'] = agg_key_name
        metrics_temp['agg_key_value'] = agg_key_value
        
        #consolidate results
        if combined_fitted_results is None:
            combined_fitted_results = metrics_temp.copy()
        else:
            combined_fitted_results = combined_fitted_results.append(metrics_temp) 
        
        # remove all nans so we can plot the charts
        combined_fitted_results.dropna(inplace=True)
        

# Linearity metric calculation - Data Extract & Fit

For linearity we follow a similar process, preaggregate the metrics and then fit a curve to it.

In [ ]:
# CQ data set is the only relevant for linearity calculation

# columns to use for the group by (apart from the aggregation key)
agg_columns = ['close_day_of_fiscal_quarter_normalised', 'close_fiscal_quarter_name']

# fields to be summarized
agg_fields = ['total_booked_net_arr', 'booked_net_arr']

# initialize to none, add. intermediate dataframes
# the goal is to calcualte the linearity for each aggregation key. First the data needs to be grouped
# at that level and the relevant fields summed up. Then the coverage calculation function is used
linearity_agg_results = None
for agg_key in agg_key_list:
    
    # the key for aggregation needs to be complimented by the day key and the quarter key
    total_agg_keys = [agg_key] + agg_columns

    #calculate the aggregation per key
    temp = cq_metrics_data.groupby(total_agg_keys,dropna=False)[agg_fields].agg('sum').reset_index().copy()
        
    # calculate coverages per metric, all are calculated using the "Pending" approach
    # this needs to be done for every iteration 
    temp['bookings_linearity'] = temp.apply(calculate_bookings_linearity, axis=1,qtd_bookings='booked_net_arr',actual_booked='total_booked_net_arr')

    # clean up / add information regarding the agg key
    temp = temp.rename({agg_key:'agg_key_value'},axis=1)
    temp['agg_key_name'] = agg_key

    #consolidate results
    if linearity_agg_results is None:
        linearity_agg_results = temp.copy()
    else:
        linearity_agg_results = linearity_agg_results.append(temp) 

# set linearity 0 to NaN
linearity_agg_results[linearity_agg_results['bookings_linearity']==0]=np.NaN

# remove NaNs from dataset
linearity_agg_results.dropna(subset=['bookings_linearity'],axis=0,inplace=True)

In [ ]:
### Curve Fitting per Cut
# for this to run I need to filter a) they aggregation level, b) the specific value at that aggregation level.
# the result will be a table like key_agg, day_metric, 

#initialization
linearity_fitted_results = None

#iterate through all the aggregated keys
for agg_key_name in linearity_agg_results.agg_key_name.unique():

    #filter temporary data
    temp_key_data = linearity_agg_results[linearity_agg_results['agg_key_name']==agg_key_name].copy()
    linearity_temp = None
    
    #iterate through the unique values of the specific aggregation
    for agg_key_value in temp_key_data.agg_key_value.unique():
        
        # filter specific value per agg temporary data
        temp_key_value_data = temp_key_data[temp_key_data['agg_key_value']==agg_key_value].copy()
                
        # set up the range of the quarter curves we want to report on
        linearity_temp = pd.DataFrame({"close_day_of_fiscal_quarter_normalised":range(0,91)})
                 
        # drop nas
        temp_linearity_data = temp_key_value_data.copy()
        temp_cuve = None

        # avoid fitting curves for cuts that do not have enough data
        if len(temp_linearity_data) > 180:     
            temp_curve = fit_curve_to_agg (temp_linearity_data,'close_day_of_fiscal_quarter_normalised','bookings_linearity') 
            linearity_temp =  linearity_temp.merge(temp_curve, how='left', on='close_day_of_fiscal_quarter_normalised')                 
            
        # add the metric detail to the dataset
        linearity_temp['agg_key_name'] = agg_key_name
        linearity_temp['agg_key_value'] = agg_key_value
        
        #consolidate results
        if linearity_fitted_results is None:
            linearity_fitted_results = linearity_temp.copy()
        else:
            linearity_fitted_results = linearity_fitted_results.append(linearity_temp) 
        
# remove all nans so we can plot the charts
linearity_fitted_results.dropna(subset=['bookings_linearity'],inplace=True)

In [ ]:
#combine with the coverage results
combined_fitted_results = combined_fitted_results.merge(linearity_fitted_results, how='left',on=['close_day_of_fiscal_quarter_normalised','agg_key_name','agg_key_value'])
combined_fitted_results.columns

# Save Quarterly METRICS to Gsheets & Drive

The final goal is to get this process to run automatically once a quarter and save the results into a Snowflake database. Until that works, we save the curve results into a tab in the Source X-Ray file.

Link to the file X-Ray Source file:

- https://docs.google.com/spreadsheets/d/1Vwu8euxRgIF3QYWK8hAbp4Vy21AlFfpDwI4MaEEiIWk/edit#gid=930691697

Link to the sheetload file:
- https://docs.google.com/spreadsheets/d/1dLevdYA8QMjpIV9irNGD8KTfKIgc3xJdrRDA2vH7M50/edit#gid=856928614

For the automatic upload on using DRIVE load from the Data Team, the column names MUST BE UPPERCASE.

Upload issue: https://gitlab.com/gitlab-data/analytics/-/issues/12157

The target folders are: 

- SS&A Coverage Fit
https://drive.google.com/drive/folders/1jfIPIbYrNO7ApTphG_0qG-IkRVvpBn3W

- SS&A Metric Data (Pre-Aggregated)
https://drive.google.com/drive/folders/19H3G2gaIBmYppDiqRxB_ChCcGx4OIZeP


In [ ]:
#https://stackoverflow.com/questions/59815620/gcloud-upload-httplib2-redirectmissinglocation-redirected-but-the-response-is-m
!pip install httplib2==0.15.0
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials

gauth = GoogleAuth()
scope = ["https://www.googleapis.com/auth/drive"]
gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name(service_file_path, scope)
drive = GoogleDrive(gauth)



In [ ]:
#### Raw data used to calculate the fitted curves
sheet_name = 'ssa_quarterly_aggregated_metrics_for_coverage'

# set the agg_key_value to string so we can index jihu out

combined_agg_results = combined_agg_results[index].dropna(subset=['agg_key_value'], how='all').copy()
combined_agg_results['agg_key_value'] = combined_agg_results['agg_key_value'].astype(str)
subset = ['agg_key_value','metric_value', 'total_booked_net_arr', 'booked_net_arr','metric_coverage']
index = ~combined_agg_results['agg_key_value'].str.contains('jihu')
for_gsheet = combined_agg_results[index].dropna(subset=subset, how='all').copy()

# nans are not welcome in the drive upload process
index = ~for_gsheet['agg_key_value'].str.contains('na')
for_gsheet = for_gsheet[index].copy()

#columns need to be upper case for the drive upload process to work
for_gsheet.columns = for_gsheet.columns.str.upper()
fields_order = ['AGG_KEY_VALUE', 'METRIC_NAME', 'CLOSE_DAY_OF_FISCAL_QUARTER_NORMALISED', 'CLOSE_FISCAL_QUARTER_NAME', 'METRIC_VALUE', 'TOTAL_BOOKED_NET_ARR', 'BOOKED_NET_ARR', 'METRIC_COVERAGE', 'AGG_KEY_NAME']
for_gsheet = for_gsheet[fields_order].copy()

# Store data in gsheets and CSV
#if save_to_gsheets_and_csv:
#    for_gsheet.to_csv('ssa_quarterly_aggregated_metrics_for_coverage_V2.csv',index=False) #ssa_quarterly_aggregated_metrics_for_coverage.csv

# upload to drive folder for autoupload -> FILE IS TO BIG FOR THIS TO WORK
#https://drive.google.com/file/d/13mjP-A831mTMlNWXNOA5EfRm_uoI5_Oo/view?usp=sharing

#if upload_to_drive_csv:
#    gfile = drive.CreateFile({'parents': [{'id': '19H3G2gaIBmYppDiqRxB_ChCcGx4OIZeP'}]})
#    gfile.SetContentFile('ssa_quarterly_aggregated_metrics_for_coverage_V2.csv')
#    gfile.Upload()

In [ ]:
# upload to the X-Ray source file, this is temporary while we do not have the data in the database
sheet_name = 'hist_coverage_fitted_curves'

subset = ['open_1plus_net_arr_coverage', 'open_3plus_net_arr_coverage','open_4plus_net_arr_coverage','rq_plus_1_open_1plus_net_arr_coverage','rq_plus_1_open_3plus_net_arr_coverage','rq_plus_1_open_4plus_net_arr_coverage', 'rq_plus_2_open_1plus_net_arr_coverage','rq_plus_2_open_3plus_net_arr_coverage','rq_plus_2_open_4plus_net_arr_coverage']
index = ~combined_fitted_results['agg_key_value'].str.contains('jihu')
for_gsheet = combined_fitted_results[index].dropna(subset=subset, how='all').copy()

for_gsheet['key_agg_day'] = for_gsheet['agg_key_value'] + '_' + for_gsheet['close_day_of_fiscal_quarter_normalised'].astype(str) 
fields_order = ['key_agg_day','agg_key_name','agg_key_value','close_day_of_fiscal_quarter_normalised','bookings_linearity','open_1plus_net_arr_coverage', 'open_3plus_net_arr_coverage','open_4plus_net_arr_coverage','rq_plus_1_open_1plus_net_arr_coverage','rq_plus_1_open_3plus_net_arr_coverage','rq_plus_1_open_4plus_net_arr_coverage', 'rq_plus_2_open_1plus_net_arr_coverage','rq_plus_2_open_3plus_net_arr_coverage','rq_plus_2_open_4plus_net_arr_coverage']

# columns in the CSV MUST BE UPPERCASE, if not, the drive process might fail
for_gsheet = for_gsheet[fields_order].copy()
for_gsheet.columns = for_gsheet.columns.str.upper()

# Store data in gsheets and CSV
if save_to_gsheets_and_csv:
    for_gsheet.to_csv('ssa_coverage_fitted_curves.csv',index=False)
#    write_to_gsheet(service_file_path, curves_sheetload_spreadsheet_id, sheet_name,for_gsheet)

# upload to drive folder for autoupload
# file link https://drive.google.com/file/d/1jM_ur0gVnhNjzQZobFfwWHGnS1jujnJg/view?usp=sharing

if upload_to_drive_csv:
    gfile = drive.CreateFile({'id':'1jM_ur0gVnhNjzQZobFfwWHGnS1jujnJg','parents': [{'id': '1jfIPIbYrNO7ApTphG_0qG-IkRVvpBn3W'}]})
    gfile.SetContentFile('ssa_coverage_fitted_curves.csv')
    gfile.Upload()
    
    
#engine.push_to_snowflake('workspace_sales',for_gsheet,'')
#pd.to_sql()

-------------------------------
# Calculation of FY Coverages & Curve Fit

In [ ]:
# Collect yearly data
sql_file = 'fy_metrics_by_day'

load_db_data = False

if load_db_data: 
    print ("Executing {} Snowflake SQL".format(sql_file))
    fy_metrics_by_day = executeScriptFromFile(sql_file+'.sql', engine)
    fy_metrics_by_day.to_csv(sql_file+'.csv',index=False)
else:
    print ("Reading Local CSV {}.csv".format(sql_file))
    fy_metrics_by_day = pd.read_csv(sql_file+'.csv')

In [ ]:
#################
##### Data Injections

# create an artificial global key to calculate a global curve for the whole company
fy_metrics_by_day['key_overall'] = 'global'

# adjust the sales_team legacy key to be sure that it won't generate duplicate with the FY23 key logic
fy_metrics_by_day['sales_team_rd_asm_level'] = 'st_rd_' + fy_metrics_by_day['sales_team_rd_asm_level']

# add a commercial and enterprise consolidated key
#add key enterprise commercial for x-ray reporting
fy_metrics_by_day['key_ent_comm'] = 'other'
fy_metrics_by_day.loc[fy_metrics_by_day['key_segment']=='large','key_ent_comm'] = 'enterprise'
fy_metrics_by_day.loc[fy_metrics_by_day['key_segment']=='pubsec','key_ent_comm'] = 'enterprise'
fy_metrics_by_day.loc[fy_metrics_by_day['key_segment']=='mid-market','key_ent_comm'] = 'commercial'
fy_metrics_by_day.loc[fy_metrics_by_day['key_segment']=='smb','key_ent_comm'] = 'commercial'

# calculate the Fiscal Years to be considered on the transformation
max_fy_year = fy_metrics_by_day.close_fiscal_year.sort_values().unique().max() 
considered_fiscal_years = [max_fy_year - 1,max_fy_year -2]
considered_fiscal_years

In [ ]:
# capture relevant data
cfy_metrics_data = fy_metrics_by_day[fy_metrics_by_day['close_fiscal_year'].isin(considered_fiscal_years)].copy()

#add fiscal year total
index = (cfy_metrics_data['close_day_of_fiscal_year_normalised']==365)
fy_total = cfy_metrics_data[index][['report_user_segment_geo_region_area_sqs_ot','close_fiscal_year','cfy_booked_net_arr']]
fy_total.rename(columns={'cfy_booked_net_arr':'total_fy_booked_net_arr'},inplace=True)
# merge total per year
cfy_metrics_data = cfy_metrics_data.merge (fy_total, how='left', on = ['report_user_segment_geo_region_area_sqs_ot','close_fiscal_year'])


### Considered metrics

For FY & 4Q we could consider Next period perspective from current point in time. Specially interesting for FY. But as of today, we only have 1 full FY to use and that is not nearly enough.

Already for current FY reporting we only have 2 valid FYs

In [ ]:
# move from wide into tall dataset
print(agg_key_list) # considered keys

### current fiscal year
variables = ['close_fiscal_year','close_day_of_fiscal_year_normalised','cfy_booked_net_arr','total_fy_booked_net_arr'] + agg_key_list
considered_metrics = ['cfy_open_1plus_net_arr','cfy_open_3plus_net_arr'] 
fy_melt = cfy_metrics_data.melt(id_vars = variables, value_vars = considered_metrics)

fy_melt = fy_melt.rename({'variable':'metric_name'}, axis=1)
fy_melt = fy_melt.rename({'value':'metric_value'}, axis=1)
fy_melt.dropna(subset=['total_fy_booked_net_arr', 'metric_value'], inplace=True)

### Calculate coverages for all business cuts

In [ ]:
# aggregation columns
agg_columns = ['metric_name','close_day_of_fiscal_year_normalised', 'close_fiscal_year']
# fields to be summarized
agg_fields = ['metric_value','total_fy_booked_net_arr', 'cfy_booked_net_arr']

# initialize to none, add. intermediate dataframes
# the goal is to calcualte the coverage for each aggregation key. First the data needs to be grouped
# at that level and the relevant fields summed up. Then the coverage calculation function is used
combined_agg_results = None
for agg_key in agg_key_list:
    
    # the key for aggregation needs to be complimented by the day key and the quarter key
    total_agg_keys = [agg_key] + agg_columns

    #calculate the aggregation per key
    temp = fy_melt.groupby(total_agg_keys,dropna=False)[agg_fields].agg('sum').reset_index().copy()
        
    # calculate coverages per metric, all are calculated using the "Pending" approach
    # this needs to be done for every iteration 
    temp['metric_coverage'] = temp.apply(calculate_to_pending_coverage, axis=1,metric='metric_value',qtd='cfy_booked_net_arr',actual='total_fy_booked_net_arr')

    # clean up / add information regarding the agg key
    temp = temp.rename({agg_key:'agg_key_value'},axis=1)
    temp['agg_key_name'] = agg_key

    #consolidate results
    if combined_agg_results is None:
        combined_agg_results = temp.copy()
    else:
        combined_agg_results = combined_agg_results.append(temp) 

### Calculate Curve Fit for FY

In [ ]:
### Curve Fitting per Cut
# for this to run I need to filter a) they aggregation level, b) the specific value at that aggregation level.
# the result will be a table like key_agg, day_metric, 

#initialization
combined_fitted_results = None

#iterate through all the aggregated keys
for agg_key_name in combined_agg_results.agg_key_name.unique():

    #filter temporary data
    temp_key_data = combined_agg_results[combined_agg_results['agg_key_name']==agg_key_name].copy()
    
    #iterate through the unique values of the specific aggregation
    for agg_key_value in temp_key_data.agg_key_value.unique():
        
        # filter specific value per agg temporary data
        temp_key_value_data = temp_key_data[temp_key_data['agg_key_value']==agg_key_value].copy()
                
        # set up the range of the quarter curves we want to report on
        metrics_temp = pd.DataFrame({"close_day_of_fiscal_year_normalised":range(0,355)})

        for metric_name in temp_key_value_data.metric_name.unique():
            
            # drop nas
            temp_metric_data = temp_key_value_data[temp_key_value_data['metric_name']==metric_name].copy()
            temp_metric_data.dropna(subset=['metric_coverage'], inplace=True)
            temp_cuve = None
            # avoid fitting curves for cuts that do not have enough data
            if len(temp_metric_data) >= 90:     
                temp_curve = fit_curve_to_agg (temp_metric_data,'close_day_of_fiscal_year_normalised','metric_coverage') 
                temp_curve.rename({'metric_coverage':metric_name+'_coverage'},inplace=1,axis=1)   
                metrics_temp =  metrics_temp.merge(temp_curve, how='left', on='close_day_of_fiscal_year_normalised')                 
            
        # add the metric detail to the dataset
        metrics_temp['agg_key_name'] = agg_key_name
        metrics_temp['agg_key_value'] = agg_key_value
        
        #consolidate results
        if combined_fitted_results is None:
            combined_fitted_results = metrics_temp.copy()
        else:
            combined_fitted_results = combined_fitted_results.append(metrics_temp) 
        
        # remove all nans so we can plot the charts
        combined_fitted_results.dropna(inplace=True)
        

In [ ]:
fy_combined_fitted_results = combined_fitted_results.copy()
fy_combined_agg_results = combined_agg_results.copy()

In [ ]:
#key_value_filter = 'large_apac_apac_korea_2. growth'
key_value_filter = 'large_emea'
metric_value_filter = 'cfy_open_1plus_net_arr'

combined_agg_results.head()
index = (combined_agg_results['agg_key_value']==key_value_filter) & (combined_agg_results['metric_name']==metric_value_filter) #& (combined_agg_results['close_fiscal_quarter_name']=='FY22-Q1')
temp = combined_agg_results[index]
sns.lineplot(data=temp,hue='close_fiscal_year', y='metric_coverage',x='close_day_of_fiscal_year_normalised')
index = (combined_fitted_results['agg_key_value']==key_value_filter) #& (combined_agg_results['close_fiscal_quarter_name']=='FY22-Q1')
temp = combined_fitted_results[index]
sns.lineplot(data=temp, y=metric_value_filter + '_coverage',x='close_day_of_fiscal_year_normalised',color='black')

In [ ]:
combined_fitted_results.columns

------------------------------
# 4Q Coverage Calculation

In [ ]:
# Collect yearly data
sql_file = 'metrics_4q_by_day'

load_db_data = True

if load_db_data: 
    print ("Executing {} Snowflake SQL".format(sql_file))
    metrics_4q_by_day = executeScriptFromFile(sql_file+'.sql', engine)
    metrics_4q_by_day.to_csv(sql_file+'.csv',index=False)
else:
    print ("Reading Local CSV {}.csv".format(sql_file))
    metrics_4q_by_day = pd.read_csv(sql_file+'.csv')

In [ ]:
#################
##### Data Injections

# create an artificial global key to calculate a global curve for the whole company
metrics_4q_by_day['key_overall'] = 'global'

# adjust the sales_team legacy key to be sure that it won't generate duplicate with the FY23 key logic
metrics_4q_by_day['sales_team_rd_asm_level'] = 'st_rd_' + metrics_4q_by_day['sales_team_rd_asm_level']

# add a commercial and enterprise consolidated key
#add key enterprise commercial for x-ray reporting
metrics_4q_by_day['key_ent_comm'] = 'other'
metrics_4q_by_day.loc[metrics_4q_by_day['key_segment']=='large','key_ent_comm'] = 'enterprise'
metrics_4q_by_day.loc[metrics_4q_by_day['key_segment']=='pubsec','key_ent_comm'] = 'enterprise'
metrics_4q_by_day.loc[metrics_4q_by_day['key_segment']=='mid-market','key_ent_comm'] = 'commercial'
metrics_4q_by_day.loc[metrics_4q_by_day['key_segment']=='smb','key_ent_comm'] = 'commercial'

In [ ]:
#filter considered quarters
index_cond = (metrics_4q_by_day['end_fiscal_quarter_date'] <= date.today() + relativedelta(months=-3)) & (metrics_4q_by_day['end_fiscal_quarter_date'] >= date.today() + relativedelta(months=-15))
cq_considered_quarters = metrics_4q_by_day[index_cond].end_fiscal_quarter_name.sort_values().unique()
cq_considered_quarters

In [ ]:
# capture relevant data
metrics_4q_data = metrics_4q_by_day[metrics_4q_by_day['end_fiscal_quarter_name'].isin(cq_considered_quarters)].copy()

# move from wide into tall dataset
print(agg_key_list) # considered keys

### current fiscal year
variables = ['start_fiscal_quarter_name','day_of_combined_4q_normalised','n4q_booked_net_arr','total_n4q_booked_net_arr'] + agg_key_list
considered_metrics = ['n4q_open_1plus_net_arr','n4q_open_3plus_net_arr'] 
melt_4q = metrics_4q_data.melt(id_vars = variables, value_vars = considered_metrics)

melt_4q = melt_4q.rename({'variable':'metric_name'}, axis=1)
melt_4q = melt_4q.rename({'value':'metric_value'}, axis=1)
melt_4q.dropna(subset=['total_n4q_booked_net_arr', 'metric_value'], inplace=True)

In [ ]:
# aggregation columns
agg_columns = ['metric_name','day_of_combined_4q_normalised', 'start_fiscal_quarter_name']
# fields to be summarized
agg_fields = ['metric_value','total_n4q_booked_net_arr', 'n4q_booked_net_arr']

# initialize to none, add. intermediate dataframes
# the goal is to calcualte the coverage for each aggregation key. First the data needs to be grouped
# at that level and the relevant fields summed up. Then the coverage calculation function is used
combined_agg_results = None
for agg_key in agg_key_list:
    
    # the key for aggregation needs to be complimented by the day key and the quarter key
    total_agg_keys = [agg_key] + agg_columns

    #calculate the aggregation per key
    temp = melt_4q.groupby(total_agg_keys,dropna=False)[agg_fields].agg('sum').reset_index().copy()
        
    # calculate coverages per metric, all are calculated using the "Pending" approach
    # this needs to be done for every iteration 
    temp['metric_coverage'] = temp.apply(calculate_to_pending_coverage, axis=1,metric='metric_value',qtd='n4q_booked_net_arr',actual='total_n4q_booked_net_arr')

    # clean up / add information regarding the agg key
    temp = temp.rename({agg_key:'agg_key_value'},axis=1)
    temp['agg_key_name'] = agg_key

    #consolidate results
    if combined_agg_results is None:
        combined_agg_results = temp.copy()
    else:
        combined_agg_results = combined_agg_results.append(temp) 

In [ ]:
### Curve Fitting per Cut
# for this to run I need to filter a) they aggregation level, b) the specific value at that aggregation level.
# the result will be a table like key_agg, day_metric, 

#initialization
combined_fitted_results = None

#iterate through all the aggregated keys
for agg_key_name in combined_agg_results.agg_key_name.unique():

    #filter temporary data
    temp_key_data = combined_agg_results[combined_agg_results['agg_key_name']==agg_key_name].copy()
    
    #iterate through the unique values of the specific aggregation
    for agg_key_value in temp_key_data.agg_key_value.unique():
        
        # filter specific value per agg temporary data
        temp_key_value_data = temp_key_data[temp_key_data['agg_key_value']==agg_key_value].copy()
                
        # set up the range of the quarter curves we want to report on
        metrics_temp = pd.DataFrame({"day_of_combined_4q_normalised":range(0,355)})

        for metric_name in temp_key_value_data.metric_name.unique():
            
            # drop nas
            temp_metric_data = temp_key_value_data[temp_key_value_data['metric_name']==metric_name].copy()
            temp_metric_data.dropna(subset=['metric_coverage'], inplace=True)
            temp_cuve = None
            # avoid fitting curves for cuts that do not have enough data
            if len(temp_metric_data) >= 90:     
                temp_curve = fit_curve_to_agg (temp_metric_data,'day_of_combined_4q_normalised','metric_coverage') 
                temp_curve.rename({'metric_coverage':metric_name+'_coverage'},inplace=1,axis=1)   
                metrics_temp =  metrics_temp.merge(temp_curve, how='left', on='day_of_combined_4q_normalised')                 
            
        # add the metric detail to the dataset
        metrics_temp['agg_key_name'] = agg_key_name
        metrics_temp['agg_key_value'] = agg_key_value
        
        #consolidate results
        if combined_fitted_results is None:
            combined_fitted_results = metrics_temp.copy()
        else:
            combined_fitted_results = combined_fitted_results.append(metrics_temp) 
        
        # remove all nans so we can plot the charts
        combined_fitted_results.dropna(inplace=True)
        

In [ ]:
n4q_combined_fitted_results = combined_fitted_results.copy()
n4q_combined_agg_results = combined_agg_results.copy()

n4q_combined_fitted_results.rename(columns={'day_of_combined_4q_normalised':'close_day_of_fiscal_year_normalised'},inplace=True)

In [ ]:
#key_value_filter = 'large_apac_apac_korea_2. growth'
key_value_filter = 'large_emea'
metric_value_filter = 'n4q_open_1plus_net_arr'

combined_agg_results.head()
index = (combined_agg_results['agg_key_value']==key_value_filter) & (combined_agg_results['metric_name']==metric_value_filter) #& (combined_agg_results['close_fiscal_quarter_name']=='FY22-Q1')
temp = combined_agg_results[index]
sns.lineplot(data=temp,hue='start_fiscal_quarter_name', y='metric_coverage',x='day_of_combined_4q_normalised')
index = (combined_fitted_results['agg_key_value']==key_value_filter) #& (combined_agg_results['close_fiscal_quarter_name']=='FY22-Q1')
temp = combined_fitted_results[index]
sns.lineplot(data=temp, y=metric_value_filter + '_coverage',x='day_of_combined_4q_normalised',color='black')

# FY & 4Q Copy to gSheet

In [ ]:
for_gsheet = fy_combined_fitted_results.copy()
for_gsheet['agg_key_value'] = for_gsheet['agg_key_value'].astype(str)
subset = ['cfy_open_1plus_net_arr_coverage', 'cfy_open_3plus_net_arr_coverage', 'agg_key_name', 'agg_key_value']

# remove jihu
index = ~for_gsheet['agg_key_value'].str.contains('jihu')
for_gsheet = for_gsheet[index].dropna(subset=subset, how='all').copy()

# remove other
index = ~for_gsheet['agg_key_value'].str.contains('other')
for_gsheet = for_gsheet[index].dropna(subset=subset, how='all').copy()

# nans are not welcome in the drive upload process
index = ~for_gsheet['agg_key_value'].str.contains('na')
for_gsheet = for_gsheet[index].copy()

#add key_agg_day
for_gsheet['key_agg_day'] = for_gsheet['agg_key_value'] + '_' + for_gsheet['close_day_of_fiscal_year_normalised'].astype(str) 

# merge the 4Q columns into the FY extract
subset = ['n4q_open_1plus_net_arr','n4q_open_3plus_net_arr', 'agg_key_name', 'agg_key_value']
for_gsheet = for_gsheet.merge(n4q_combined_fitted_results, how='left',on=['agg_key_value','agg_key_name', 'close_day_of_fiscal_year_normalised'])

print(for_gsheet.columns)

# define the field order
fields_order = ['key_agg_day', 'agg_key_value', 'close_day_of_fiscal_year_normalised', 'cfy_open_1plus_net_arr_coverage', 'cfy_open_3plus_net_arr_coverage','n4q_open_1plus_net_arr_coverage','n4q_open_3plus_net_arr_coverage']
for_gsheet = for_gsheet[fields_order].copy()

#columns need to be upper case for the drive upload process to work
for_gsheet.columns = for_gsheet.columns.str.upper()
for_gsheet.fillna(0,inplace=True)

In [ ]:
# Store data in gsheets and CSV
if save_to_gsheets_and_csv:
    for_gsheet.to_csv('fy_coverage_fitted_curves.csv',index=False) #fy_coverage_fitted_curves.csv
    curves_sheetload_spreadsheet_id = "1Vwu8euxRgIF3QYWK8hAbp4Vy21AlFfpDwI4MaEEiIWk"
    sheet_name = "next_4q_hist_cov_example"
    write_to_gsheet(service_file_path, curves_sheetload_spreadsheet_id, sheet_name,for_gsheet)

--------------------------------

# Test Code

The following snippets were used during development to test the approach

In [ ]:
# Test that results make sense
cq_plus2_metrics_data.groupby(['close_fiscal_quarter_name','close_day_of_fiscal_quarter_normalised'])['rq_plus_2_open_1plus_net_arr'].sum()

In [ ]:
######## Test that the totals work. E.g. FY22Q4 Large = 29043057
# combined_df[(combined_df['close_day_of_fiscal_quarter_normalised']==40) & (combined_df['metric']=='open_1plus_net_arr')].groupby(by=['key_segment','close_fiscal_quarter_name']).total_booked_net_arr.sum()

####### Test that the coverage curve makes sense for one metric
index = (combined_df['key_segment']=='large') & (combined_df['metric_name']=='open_1plus_net_arr') & (combined_df['close_fiscal_quarter_name']=='FY22-Q1')
temp = combined_df[index].groupby(['close_day_of_fiscal_quarter_normalised','key_segment']).agg({'total_booked_net_arr':'sum','metric_value':'sum','booked_net_arr':'sum'}).reset_index()

# calculate coverages per metric, all are calculated using the "Pending" approach
# this needs to be done for every iteration 
temp['metric_coverage'] = temp.apply(calculate_to_pending_coverage, axis=1,metric='metric_value',qtd='booked_net_arr',actual='total_booked_net_arr')

sns.lineplot(data=temp,hue='key_segment', y='metric_coverage',x='close_day_of_fiscal_quarter_normalised')

#### Test #1 that combined pre-grouped dataset has reasonable values
combined_agg_results.head()
index = (combined_agg_results['agg_key_value']=='large') & (combined_agg_results['metric_name']=='open_1plus_net_arr') & (combined_agg_results['close_fiscal_quarter_name']=='FY22-Q1')
temp = combined_agg_results[index]
temp.head()
sns.lineplot(data=temp,hue='agg_key_value', y='metric_coverage',x='close_day_of_fiscal_quarter_normalised')


In [ ]:

#### Test #2 that combined pre-grouped dataset has reasonable values

#key_value_filter = 'large_apac_apac_korea_2. growth'
key_value_filter = 'large'
metric_value_filter = 'rq_plus_2_open_1plus_net_arr'

combined_agg_results.head()
index = (combined_agg_results['agg_key_value']==key_value_filter) & (combined_agg_results['metric_name']==metric_value_filter) #& (combined_agg_results['close_fiscal_quarter_name']=='FY22-Q1')
temp = combined_agg_results[index]
sns.lineplot(data=temp,hue='close_fiscal_quarter_name', y='metric_coverage',x='close_day_of_fiscal_quarter_normalised')
index = (combined_fitted_results['agg_key_value']==key_value_filter) #& (combined_agg_results['close_fiscal_quarter_name']=='FY22-Q1')
temp = combined_fitted_results[index]
sns.lineplot(data=temp, y=metric_value_filter + '_coverage',x='close_day_of_fiscal_quarter_normalised',color='black')

In [ ]:
#### Test #2a that combined pre-grouped dataset has reasonable values

key_value_filter = 'large_apac_apac_korea_2. growth'
metric_value_filter = 'bookings_linearity'

combined_agg_results.head()
index = (combined_agg_results['agg_key_value']==key_value_filter) & (combined_agg_results['metric_name']==metric_value_filter) #& (combined_agg_results['close_fiscal_quarter_name']=='FY22-Q1')
temp = combined_agg_results[index]
sns.lineplot(data=temp,hue='close_fiscal_quarter_name', y='metric_coverage',x='close_day_of_fiscal_quarter_normalised')
combined_fitted_results.head()
index = (combined_fitted_results['agg_key_value']==key_value_filter) #& (combined_agg_results['close_fiscal_quarter_name']=='FY22-Q1')
temp = combined_fitted_results[index]
sns.lineplot(data=temp, y=metric_value_filter,x='close_day_of_fiscal_quarter_normalised',color='black')

In [ ]:
#### Test #3 that combined pre-grouped dataset has reasonable values
combined_fitted_results.head()
index = (combined_fitted_results['agg_key_value']=='large_amer_west')
temp = combined_fitted_results[index]
temp.head()
sns.lineplot(data=temp,hue='agg_key_value', y='open_1plus_net_arr_coverage',x='close_day_of_fiscal_quarter_normalised')

In [ ]:
combined_fitted_results

In [ ]:
combined_agg_results[combined_agg_results['agg_key_value']=='pubsec']

In [ ]:
len(combined_fitted_results)